In [32]:
# Imports
import numpy as np
import pandas as pd
import gspread as gs
from tqdm import tqdm
import traceback
import os
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
import firebase_admin
from firebase_admin import credentials, firestore
from firebase_admin import auth
import warnings
warnings.filterwarnings("ignore")


# Store Errors (if any) when updating the trips doc
script_path = os.path.abspath('Update Firestore records.ipynb')
script_dir = os.path.split(script_path)[0]
error_path = r"Errors"
abs_error_path = os.path.join(script_dir, error_path)
#print(abs_error_path)


# Authenticate Firestore
try:
    app = firebase_admin.get_app()
except ValueError as e:
    cred = credentials.Certificate('C:/Users/leemn/OneDrive/Documents/Service Account/accountkey_firebase.json')
    firebase_admin.initialize_app(cred)
db = firestore.client() 


# Authenticate BQ
file = 'C:/Users/leemn/OneDrive/Documents/Service Account/cb-prod-297913-d207de781afd.json'
creds = Credentials.from_service_account_file(file)
# Authenticate Google sheets
gc = gs.service_account(filename=file)


# Import Google Sheet file and save as pandas dataframe
gsheet_file = 'https://docs.google.com/spreadsheets/d/1I7yAvQe3xkutUkPuXBw9a8DmI8IVoKrzIa-1tibMR3U'
sheet_name = 'Orders'
sh = gc.open_by_url(gsheet_file)
ws = sh.worksheet(sheet_name)
data_orders = pd.DataFrame(ws.get_all_records())


# lookup trip_id from the imported order_id list (ensure order_id field is in the file)
order_list = list(data_orders.order_id.unique())
p = tuple(order_list)
query_orders = '''
SELECT DISTINCT order_id, order_no, trip_id
FROM `cb-prod-297913.cb_prod_analytics.cb_end_orders_in_trip` 
WHERE order_id IN {}
GROUP BY 1,2,3
'''.format(p)

data_trips = pd.read_gbq(query_orders, project_id='cb-prod-297913', 
                         dialect='standard', 
                         #progress_bar_type='tqdm_notebook',
                         credentials=creds)


# set driver_earnings (wallet cost) to zero
data_trips['driver_earnings'] = 0.0

In [ ]:
batch = db.batch()

for trip_id, driver_earnings in tqdm(zip(data_trips.trip_id, data_trips.driver_earnings)):
    try:
        docs = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J').collection('trips').document(trip_id)
        #print(trip_id)
        #print(driver_earnings)
        #print(predictedKM)
        batch.update(docs, {u'wallet.cost': driver_earnings})
        #batch.update(docs, {u'distance': str(predictedKM)})
        batch.commit()
    except Exception as e:
        log = open(abs_error_path+'\\'+str(trip_id)+".txt", "a")
        log.write(':\n')
        traceback.print_exc(file=log)      
        log.write('\n')
        log.close()    